In [1]:
import time
import boto3
import sagemaker
import urllib
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# 获取当前AWS region和notebook所绑定的role
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [3]:
# One has to set up a S3 bucket before running the code below
bucket = 'tommy-lab'
prefix = 'beijing-housing-data'

In [4]:
cwd = os.getcwd()
print(cwd)

/home/ec2-user/SageMaker/706-Final-Project/10_Code


In [5]:
X = pd.read_csv("/home/ec2-user/SageMaker/706-Final-Project/20_Cleaned_Data/prepared_data_tommy.csv" )

In [6]:
# # 可以使用 SageMaker session 来上传下载数据，这里把S3的训练数据下载到当前目录
# sm_session = sagemaker.Session()
# sm_session.download_data("./", bucket, prefix + 'train.csv')
# sm_session.download_data("./", bucket, prefix + 'test.csv')

In [7]:
# 通过describe来对查看一下训练数据feature的构成和分布
X.head()

,Price/Square Meter,Housing_ID,Community,Livable Area,Price,Year_Built,Facing_East,Facing_North,Facing_South,Facing_West,Decoration_Level,Floor_Level,Floor_Plan
0,38544.0,1,中关村南大街甲3号 2室1厅 56.3平米,56.30,217.0,1965,0,0,1,0,1,2,1
1,33667.0,2,中关村南大街甲3号 2室1厅 55.1平米,55.10,185.5,1965,0,0,1,0,3,1,1
2,91429.0,3,民族大学南路19号院 2室1厅 70平米,70.00,640.0,1993,0,0,1,0,1,3,1
3,102752.0,4,民族大学南路19号院 2室1厅 98.49平米,98.49,1012.0,2000,0,0,1,0,1,4,1
4,98847.0,5,民族大学南路19号院 2室1厅 60.7平米,60.70,600.0,1991,0,0,1,0,3,4,1


In [8]:
# Extract the column Price/Square Meter from X
y = X['Price/Square Meter']

In [9]:
y.head()

0     38544.0
1     33667.0
2     91429.0
3    102752.0
4     98847.0
Name: Price/Square Meter, dtype: float64

In [10]:
# Drop the column Price/Square Meter in X
X.drop(['Price/Square Meter'], axis=1, inplace=True)

In [11]:
X.head()

,Housing_ID,Community,Livable Area,Price,Year_Built,Facing_East,Facing_North,Facing_South,Facing_West,Decoration_Level,Floor_Level,Floor_Plan
0,1,中关村南大街甲3号 2室1厅 56.3平米,56.30,217.0,1965,0,0,1,0,1,2,1
1,2,中关村南大街甲3号 2室1厅 55.1平米,55.10,185.5,1965,0,0,1,0,3,1,1
2,3,民族大学南路19号院 2室1厅 70平米,70.00,640.0,1993,0,0,1,0,1,3,1
3,4,民族大学南路19号院 2室1厅 98.49平米,98.49,1012.0,2000,0,0,1,0,1,4,1
4,5,民族大学南路19号院 2室1厅 60.7平米,60.70,600.0,1991,0,0,1,0,3,4,1


In [12]:
# 拆分原始训练数据为四份，X_train, X_valid, y_train, y_valid (We still have community name here)
X_train_with_community, X_valid_with_community, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [13]:
# Make a new X_train, y_train (without community name here)
X_train = X_train_with_community.drop(columns = ['Community'])
X_valid = X_valid_with_community.drop(columns = ['Community'])

In [14]:
X_train_xgb= pd.concat([y_train,X_train], axis=1)
X_train_xgb.to_csv('./X_train.csv',header=False,index=False)

In [15]:
X_valid_xgb= pd.concat([y_valid,X_valid], axis=1)
X_valid_xgb.to_csv('./X_valid.csv',header=False,index=False)

In [16]:
X_valid_xgb_new= pd.concat([y_valid,X_valid_with_community], axis=1)

In [17]:
sm_session = sagemaker.Session()
train_data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'train')
validation_data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'validation')
sm_session.upload_data('./X_train.csv',bucket,prefix)
sm_session.upload_data('./X_valid.csv',bucket,prefix)

's3://tommy-lab/beijing-housing-data/X_valid.csv'

In [18]:
# 我们使⽤SageMaker内置的XGBoost来训练，⾸先要获取这个算法的container
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region, 'xgboost', repo_version='0.90-2')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [19]:
# 设置算法超参
hyperparameters = {
"max_depth":"5",
"eta":"0.5",
"early_stopping_rounds":"5",
"eval_metric":"rmse",
"num_round":"30",
"objective":"reg:linear"}

In [20]:
# 训练使⽤哪种EC2实例来完成
instance_type = 'ml.m4.xlarge'

In [21]:
# 模型输出⽬录
output_path = 's3://{}/{}/output'.format(bucket, prefix)

In [22]:
# 训练输⼊数据的类型
content_type = "csv"

In [23]:
# 设置训练任务的名字
job_name = 'xgb-housing-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print("Training job", job_name)

Training job xgb-housing-2020-11-21-15-18-31


In [24]:
# 借助 Managed Spot Training 功能，⼤幅降低训练成本
train_use_spot_instances = True
train_max_run = 3600
train_max_wait = 3600 if train_use_spot_instances else None
checkpoint_s3_uri = ('s3://{}/{}/checkpoints/{}'.format(bucket, prefix,
job_name)
if train_use_spot_instances else None)

In [25]:
# 在SageMaker⾥的使⽤任何算法来训练都要先⽣成⼀个 estimator 对象
xgb = sagemaker.estimator.Estimator(container,
role,
hyperparameters=hyperparameters,
train_instance_count=1,
train_instance_type=instance_type,
train_volume_size=5, # 5 GB
output_path=output_path,
sagemaker_session=sagemaker.Session(),
train_use_spot_instances=train_use_spot_instances,
train_max_run=train_max_run,
train_max_wait=train_max_wait,
checkpoint_s3_uri=checkpoint_s3_uri
);

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [26]:
# 设置训练数据的 data channel
train_data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'X_train.csv')
validation_data_location = 's3://{}/{}/{}'.format(bucket, prefix,
'X_valid.csv')
train_channel = sagemaker.session.s3_input(train_data_location,
content_type='csv')
valid_channel = sagemaker.session.s3_input(validation_data_location,
content_type='csv')
data_channels = {'train': train_channel, 'validation': valid_channel}

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [27]:
%%time
# 调⽤ fit 来训练
xgb.fit(inputs=data_channels, job_name=job_name, logs=True)

2020-11-21 15:18:33 Starting - Starting the training job...
2020-11-21 15:18:34 Starting - Launching requested ML instances.........
2020-11-21 15:20:06 Starting - Preparing the instances for training......
2020-11-21 15:21:25 Downloading - Downloading input data...
2020-11-21 15:21:53 Training - Downloading the training image...
2020-11-21 15:22:33 Uploading - Uploading generated training model
2020-11-21 15:22:33 Completed - Training job completed
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter eval_metric value rmse to Json.
Returning the value itself
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determin

In [28]:
%%time
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# xgb = sagemaker.estimator.Estimator(container,
#                                     role=sagemaker.get_execution_role(), 
#                                     instance_count= 1, # make sure you have limit set for these instances
#                                     instance_type='ml.m4.xlarge', 
#                                     output_path='s3://{}/{}/output'.format(bucket, prefix),
#                                     sagemaker_session=sagemaker.Session())

# # # 设置算法超参
# # hyperparameters = {
# # "max_depth":"5",
# # "eta":"0.5",
# # "early_stopping_rounds":"5",
# # "eval_metric":"rmse",
# # "num_round":"30"
# # "objective"="reg:linear",
# }


hyperparameter_ranges = {'max_depth':IntegerParameter(1,10),
                         'alpha': ContinuousParameter(0, 100),
                         'min_child_weight': ContinuousParameter(1, 5),
                         'eta': ContinuousParameter(0.2, 0.7),  
                         'num_round': IntegerParameter(10,200)
                         }


objective_metric_name = 'validation:rmse'
objective_type = 'Minimize'

tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=3, # Set this to 10 or above depending upon budget & available time.
                            max_parallel_jobs=1,
                            objective_type=objective_type,
                            early_stopping_type='Auto')

tuner.fit(inputs=data_channels, include_cls_metadata=False)
tuner.wait()

...................................................................................................................................................................!
!
CPU times: user 772 ms, sys: 66.7 ms, total: 838 ms
Wall time: 13min 45s


In [29]:
%%time
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

CPU times: user 63.1 ms, sys: 3.56 ms, total: 66.6 ms
Wall time: 1.14 s


'Completed'

In [30]:
%%time
from pprint import pprint
from sagemaker.analytics import HyperparameterTuningJobAnalytics

tuner_analytics = HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.name, sagemaker_session=sagemaker.Session())

df_tuning_job_analytics = tuner_analytics.dataframe()

# Sort the tuning job analytics by the final metrics value
df_tuning_job_analytics.sort_values(
    by=['FinalObjectiveValue'],
    inplace=True,
    ascending=False if tuner.objective_type == "Maximize" else True)

# Show detailed analytics for the top 20 models
df_tuning_job_analytics.head(20)

CPU times: user 25.6 ms, sys: 0 ns, total: 25.6 ms
Wall time: 88.6 ms


,alpha,eta,max_depth,min_child_weight,num_round,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,3.642550,0.608976,8.0,4.653224,90.0,sagemaker-xgboost-201121-1522-003-54778498,Completed,961.213989,2020-11-21 15:34:32+00:00,2020-11-21 15:35:53+00:00,81.0
2,13.800875,0.591418,3.0,4.386295,194.0,sagemaker-xgboost-201121-1522-001-4e394c4b,Completed,1186.500000,2020-11-21 15:25:26+00:00,2020-11-21 15:26:50+00:00,84.0
1,74.896090,0.209893,4.0,3.722760,10.0,sagemaker-xgboost-201121-1522-002-08cecf97,Completed,8359.639648,2020-11-21 15:30:05+00:00,2020-11-21 15:31:57+00:00,112.0


In [31]:
attached_tuner = HyperparameterTuner.attach(tuner.latest_tuning_job.name, sagemaker_session=sagemaker.Session())
best_training_job = attached_tuner.best_training_job()

In [32]:
from sagemaker.estimator import Estimator
algo_estimator = Estimator.attach(best_training_job)

best_algo_model = algo_estimator.create_model(env={'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT':"text/csv"})


2020-11-21 15:35:53 Starting - Preparing the instances for training
2020-11-21 15:35:53 Downloading - Downloading input data
2020-11-21 15:35:53 Training - Training image download completed. Training in progress.
2020-11-21 15:35:53 Uploading - Uploading generated training model
2020-11-21 15:35:53 Completed - Training job completed


In [33]:
# %%time
# batch_output = "s3://{}/{}/batch-out/".format(bucket,prefix)
# batch_input = "s3://{}/{}/batch-in/{}".format(bucket,prefix,batch_X_file)

# xgb_transformer = best_algo_model.transformer(instance_count=1,
#                                        instance_type='ml.m4.xlarge',
#                                        strategy='MultiRecord',
#                                        assemble_with='Line',
#                                        output_path=batch_output)


# xgb_transformer.transform(data=batch_input,
#                          data_type='S3Prefix',
#                          content_type='text/csv',
#                          split_type='Line')
# xgb_transformer.wait(logs=False)

In [34]:
%%time
# 部署模型

xgb_predictor = best_algo_model.deploy(initial_instance_count=1,
                         instance_type='ml.m4.xlarge')


-----------------!CPU times: user 247 ms, sys: 24.5 ms, total: 272 ms
Wall time: 8min 32s


In [38]:
%%time
from sagemaker.predictor import csv_serializer
# 创建 predictor
xgb_predictor=sagemaker.predictor.RealTimePredictor(
"sagemaker-xgboost-2020-11-21-15-36-33-324", # 这个名字就是上图中蓝⾊⽅框中endpoint的name
sagemaker_session=sm_session,
serializer=csv_serializer,
content_type='csv')
# 调⽤predictor的predict⽅法做推理
# xgb_predictor.predict(X_valid.values[0]).decode('utf-8')
result = xgb_predictor.predict(X_valid.values).decode('utf-8')

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
content_type is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


CPU times: user 194 ms, sys: 46.2 ms, total: 241 ms
Wall time: 1.79 s


In [39]:
result

'40483.71484375,84208.0390625,56475.28515625,35975.60546875,97039.484375,36734.41015625,74010.96875,24873.30859375,53419.92578125,32178.546875,92192.640625,31349.822265625,56541.5859375,87494.265625,52567.1328125,83786.515625,86705.4375,48659.921875,58426.70703125,94004.25,89481.40625,46570.7421875,95850.953125,49214.17578125,53198.8359375,27878.3203125,61974.50390625,50743.8203125,53932.95703125,79878.90625,37489.40625,63547.96875,31332.427734375,38281.07421875,47605.125,45016.1796875,57973.83984375,76577.03125,36640.23828125,75101.3984375,96339.3984375,40156.80859375,106859.7109375,73965.828125,40538.8671875,83593.6640625,46014.94140625,50924.85546875,59770.18359375,35464.6875,48074.125,50701.125,36654.453125,79902.8203125,72701.53125,77428.375,52734.15234375,56700.6953125,90189.75,81199.21875,44422.75,30930.365234375,61225.4296875,60904.08984375,71477.8046875,67138.5390625,39478.3515625,84289.21875,59260.69140625,54426.79296875,34744.41015625,30739.0859375,48388.7890625,67659.164062

In [51]:
result2 = result.split(',')
result_list = []

for i in result2:
    i = round(float(i),2)
    result_list.append(i)

X_valid_xgb_new['Prediction'] = result_list
X_valid_xgb_new['Error'] = abs(X_valid_xgb_new['Price/Square Meter'] - X_valid_xgb_new['Prediction'])

first30 = X_valid_xgb_new.head(1000)

In [52]:
first30 = first30.rename(columns = {'Livable Area': 'Livable_Area'})

In [63]:
import ipywidgets as widgets
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt


# ALL_1 = 'ALL (Livable Area)'
ALL_2 = 'ALL (Decoration Level)'
ALL_3 = 'ALL (Year Built)'

# def unique_sorted_values_plus_Livable_Area(array):
#     unique = array.unique().tolist()
#     unique.sort()
#     unique.insert(0, ALL_1)
#     return unique

def unique_sorted_values_plus_Decoration_Level(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL_2)
    return unique

def unique_sorted_values_plus_Year_Built(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL_3)
    return unique


output = widgets.Output()
plot_output = widgets.Output()

# dropdown_Livable_Area = widgets.Dropdown(options = unique_sorted_values_plus_Livable_Area(first30.Livable_Area))
dropdown_Decoration_Level = widgets.Dropdown(options = unique_sorted_values_plus_Decoration_Level(first30.Decoration_Level))
dropdown_Year_Built = widgets.Dropdown(options = unique_sorted_values_plus_Year_Built(first30.Year_Built))


def common_filtering(Year_Built, Decoration_Level):
    output.clear_output()
    plot_output.clear_output()
    
    if (Decoration_Level == ALL_2) & (Year_Built == ALL_3) :
        common_filter = first30
    elif (Decoration_Level == ALL_2):
        common_filter = first30[first30.Year_Built == Year_Built]
    elif (Year_Built == ALL_3):
        common_filter = first30[first30.Decoration_Level == Decoration_Level]
    else:
        common_filter = first30[(first30.Decoration_Level == Decoration_Level) & 
                                  (first30.Year_Built == Year_Built)]
    
    with output:
        display(common_filter)
        
    with plot_output:
        sns.kdeplot(common_filter['Price/Square Meter'], shade=True)
        plt.show()
    
        
def dropdown_Year_Built_eventhandler(change):
    common_filtering(change.new, dropdown_Decoration_Level.value)
    
def dropdown_Decoration_Level_eventhandler(change):
    common_filtering(dropdown_Year_Built.value, change.new)
    
dropdown_Year_Built.observe(dropdown_Year_Built_eventhandler, names='value')
dropdown_Decoration_Level.observe(dropdown_Decoration_Level_eventhandler, names='value')

In [69]:
item_layout = widgets.Layout(margin='0 0 50px 0')
input_widgets = widgets.HBox([dropdown_Year_Built, dropdown_Decoration_Level],layout=item_layout)

tab = widgets.Tab([output, plot_output])
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'KDE Plot')

dashboard = widgets.VBox([input_widgets, tab])
display(dashboard)

In [ ]:
# sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)